# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import os

# Define Source

In [2]:
link = 'https://www.autism.org/autism-prognosis/'

# Read 100 questions list

In [8]:
file = 'Excel/' + os.listdir('Excel')[-1]
df = pd.read_excel(file,engine='openpyxl')

df[link]=np.nan

# Scrape QA pairs from website

In [4]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [5]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [18]:
x = soup.find('div',{'class':'fusion-text fusion-text-1'}).text

In [16]:
l1=[]
for i in soup.find_all('h2'):
    l1.append(i.text.strip())
l1=l1[:-1]
l1

['Prognosis for Autism',
 'What is Autism?',
 'Treatments influence prognosis',
 'Early intervention yields better results']

In [17]:
l2=[]
for i in range(len(l1)):
    print(l1[i])
    print('-'*50)
    start = x.index(l1[i])

    if(i!=len(l1)-1):
        end = x.index(l1[i+1])
    if(i==len(l1)-1):
        end = len(x)
       
    ans = x[start:end]
    ans = ans.split(l1[i])[1]
    ans = ans.strip()
    print(ans)
    print('*'*100)
    
    l2.append(ans)

Prognosis for Autism
--------------------------------------------------
If your loved one has been diagnosed with autism spectrum disorder, you may wonder about the likely course of their condition. Will they improve? Will challenging behaviors stop or increase? What can you do to support their development and growth over time? Because every individual with autism is different, there are no universal answers to these questions. However, the prognosis for people with autism has improved over the last two decades as more treatments have been identified and tested.
****************************************************************************************************
What is Autism?
--------------------------------------------------
Autism is a neurodevelopmental disorder that begins at birth or within the first two-and-a-half years of life. Autism is a spectrum disorder, meaning that it can appear in a range of forms and severity levels. The prognosis for a child with autism depends on the 

# Check which questions are similar

In [21]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('autism spectrum disorder','asd',sent) # Compress term
    sent = re.sub('autism','asd',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [22]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [23]:
def sim(li):
    for c1,i in enumerate(li):
        i2 = clean_text(i)
        list1 = i2.split()

        temp1 = 0.0
        temp2 = ''
        temp3 = ''
        temp4 = 0
        temp5 = 0

        for c2,j in enumerate(df['Question']):
            j2 = clean_text(j)
            list2 = j2.split()

            sim = jaccard_similarity(list1, list2)

            if(sim>temp1):
                temp1 = sim
                temp2 = j
                temp3 = i
                temp4 = c2
                temp5 = c1

        if(temp1>0):
            print('Website --> ',temp3,'(Index {})'.format(temp5))
            print('100 questions list --> ',temp2,'(Index {})'.format(temp4))
            print(temp1)
            print('*'*100)

In [24]:
sim(l1)

Website -->  Prognosis for Autism (Index 0)
100 questions list -->  What is the prognosis of children with Autism? (Index 26)
0.6666666666666666
****************************************************************************************************
Website -->  What is Autism? (Index 1)
100 questions list -->  What is Autism? (Index 4)
1.0
****************************************************************************************************
Website -->  Treatments influence prognosis (Index 2)
100 questions list -->  What is the prognosis of children with Autism? (Index 26)
0.2
****************************************************************************************************
Website -->  Early intervention yields better results (Index 3)
100 questions list -->  What are early red flags of ASD in toddlers? (Index 2)
0.1111111111111111
****************************************************************************************************


In [ ]:
# Website, 100Q
# 0, 26
# 1, 4

In [28]:
for i,j in enumerate(df['Question']):
    if('treat' in j):
        print(i,j)

15 Are there other therapies that are useful in treatment of behavioral symptoms in Autistic children?
65 Are medications useful in treating the behavioral problems of Autistic children?
66 What types of medications are used to treat autistic children?
68 Are antipsychotic medications helpful in the treatment of autism?
69 I’ve heard that anti-seizure medication can be helpful in treating some behavior problems associated with autism. Is that true?
70 How are Anxiety and depression treated in an autistic person?
71 What medications are used to treat seizures in autistic children?
72 What are some treatments for involuntary movements, vocalizations and Tourette’s Syndrome associated with Autism?
73 Is there any harm in trying unproven treatments on my child?
75 What are some common dietary interventions used to treat Autism?
76 Is Secretin helpful in treating the symptoms of Autism?
77 Is Naltrexone helpful in treating autism?


In [29]:
# Directly add answer to existing dataframe
l4 = [[0, 26],[1, 4]]
for i in l4:
    df[link].loc[i[1]] = l2[i[0]]

C:\Users\aparg\anaconda3\envs\ASD_scraping\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [30]:
l1

['Prognosis for Autism',
 'What is Autism?',
 'Treatments influence prognosis',
 'Early intervention yields better results']

In [31]:
# update questions list
f = [i[0] for i in l4]
f = set(range(len(l1)))-set(f)
l1=[l1[i] for i in f]
l1

['Treatments influence prognosis', 'Early intervention yields better results']

In [32]:
# select answers to questions in list 'l'
l2 = [l2[i] for i in f]

In [33]:
# Create new dataframe with QA pairs
df2 = pd.DataFrame(zip(l1,l2),columns=['Question',link])
df2

,Question,https://www.autism.org/autism-prognosis/
0,Treatments influence prognosis,"Many researchers, clinicians, and parents have..."
1,Early intervention yields better results,Age at intervention can impact long-term outco...


In [36]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 = df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)

In [37]:
df3.iloc[:,[0,-1]].dropna()

,Question,https://www.autism.org/autism-prognosis/
4,What is Autism?,Autism is a neurodevelopmental disorder that b...
26,What is the prognosis of children with Autism?,If your loved one has been diagnosed with auti...
30,Early intervention yields better results,Age at intervention can impact long-term outco...
31,Treatments influence prognosis,"Many researchers, clinicians, and parents have..."


In [38]:
df3.shape

(119, 6)

# Save Output

In [39]:
df3.to_excel('Excel/Source2_4.xlsx',index=False)